<a href="https://colab.research.google.com/github/taravatp/roadLane_InstanceSegmentation/blob/main/models/Hnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class HnetBlock(nn.Module):
  def __init__(self,in_channels,out_channels):
    super(HnetBlock,self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(in_channels,out_channels,3,padding=1),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
    )

    self.conv2 = nn.Sequential(
      nn.Conv2d(out_channels,out_channels,3,padding=1),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
    )

    self.pool = nn.MaxPool2d(2)

  def forward(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.pool(x)
    return x


In [ ]:
class Hnet(nn.Module):
  def __init__(self):
    super(Hnet,self).__init__()

    self.block1 = HnetBlock(3,16)
    self.block2 = HnetBlock(16,32)
    self.block3 = HnetBlock(32,64)
    self.fc = nn.Sequential(
        nn.Linear(64 * 16 * 8, 1024),
        nn.ReLU(),
        nn.Linear(1024,6)
    )

  def forward(self,x):
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = x.view(-1,64*16*8) #drop the first dimension
    x = self.fc(x)
    return x

In [ ]:
if __name__ == '__main__':
    hnet = Hnet()
    test_input = torch.rand(8, 3, 64, 128, dtype=torch.float32)
    output = hnet(test_input)
    print(output.size())

torch.Size([8, 6])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
hnet = Hnet()
print(hnet)

Hnet(
  (block1): HnetBlock(
    (conv1): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): HnetBlock(
    (conv1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReL